In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import pandas as pd

os.chdir(os.path.abspath('..'))


In [2]:
from dinov2.data.datasets.satlas import SatlasDataset

root = '/data/panopticon/datasets/satlas'

satlas_ds = SatlasDataset(root=root, normalize=True, full_spectra=True)

In [37]:
set(satlas_ds.s2_bandname_map.values())

{'b05', 'b06', 'b07', 'b08', 'b11', 'b12', 'tci'}

In [52]:
sample = satlas_ds[0]
sample

s1: 2
l9: 11
torch.Size([2, 2])
torch.Size([11, 2])
[[tensor([[-1., -1.]]), tensor([[-2., -1.]])], [tensor([[442.7355,  10.8443]]), tensor([[482.3008,  39.8427]]), tensor([[560.9277,  38.5178]]), tensor([[654.2978,  25.2545]]), tensor([[864.6171,  20.1719]]), tensor([[1608.4401,   60.7333]]), tensor([[2200.9832,  130.8720]]), tensor([[1374.1533,   14.9113]]), tensor([[593.9452, 111.8967]]), tensor([[10838.8330,   526.2402]]), tensor([[12023.3994,   644.5646]])]] 2


[{'imgs': [[tensor([[[0.1118, 0.1275, 0.1431,  ..., 0.0725, 0.0725, 0.0647],
             [0.1235, 0.1353, 0.1431,  ..., 0.0608, 0.0608, 0.0569],
             [0.1314, 0.1392, 0.1431,  ..., 0.0412, 0.0373, 0.0333],
             ...,
             [0.0176, 0.0255, 0.0294,  ..., 0.0961, 0.0961, 0.0922],
             [0.0216, 0.0255, 0.0294,  ..., 0.1039, 0.1039, 0.0961],
             [0.0216, 0.0216, 0.0255,  ..., 0.1078, 0.1078, 0.1000]]]),
    tensor([[[ 0.0059,  0.0020, -0.0176,  ..., -0.0647, -0.0725, -0.0490],
             [-0.0059, -0.0059, -0.0216,  ..., -0.0843, -0.0961, -0.0725],
             [-0.0137, -0.0137, -0.0255,  ..., -0.0882, -0.1078, -0.1039],
             ...,
             [-0.1824, -0.2098, -0.2137,  ..., -0.0490, -0.0569, -0.0529],
             [-0.1667, -0.1902, -0.2020,  ..., -0.0490, -0.0529, -0.0569],
             [-0.1627, -0.1745, -0.1824,  ..., -0.0451, -0.0490, -0.0569]]])]],
  'chn_ids': [[tensor([[-1., -1.]]), tensor([[-2., -1.]])]]},
 {'imgs': [[tensor([[[

In [43]:
len(sample[1]['imgs'][0]), len(sample[1]['chn_ids'][0])

(9, 9)

In [47]:
for i in range(len(sample[1]['chn_ids'][0])):
    print(sample[1]['chn_ids'][0][i].shape)

torch.Size([1, 2])
torch.Size([1, 2])
torch.Size([1, 2])
torch.Size([1, 2])
torch.Size([1, 2])
torch.Size([1, 2])
torch.Size([1, 2])
torch.Size([1, 2])
torch.Size([1, 2])


In [48]:
for i in range(len(sample[1]['imgs'][0])):
    print(sample[1]['imgs'][0][i].shape)

torch.Size([1, 512, 512])
torch.Size([1, 512, 512])
torch.Size([1, 512, 512])
torch.Size([1, 512, 512])
torch.Size([1, 512, 512])
torch.Size([1, 512, 512])
torch.Size([1, 512, 512])
torch.Size([1, 512, 512])
torch.Size([1, 512, 512])


## With loader & panopticon on device

In [61]:
from omegaconf import OmegaConf
from dinov2.eval.setup import setup_logger
from dinov2.data.loaders import make_dataset
from dinov2.models.panopticon import ChnAttnPatchEmb
global logger

patch_emb_cfg = {'patch_size': 14, 'embed_dim': 256}
chnAttnBlock_cfg = {'id_attn_block' :'ChnAttnBlockSimple', 'dim' : 256, 'num_heads':1}
chnEmb_cfg = {'embed_dim':256}

pan = ChnAttnPatchEmb(patch_emb_cfg=patch_emb_cfg, chnAttnBlock_cfg=chnAttnBlock_cfg, chnEmb_cfg=chnEmb_cfg)


logger = setup_logger('dinov2', to_sysout=True, simple_prefix=True)

cfg = OmegaConf.load('notebooks/explore_crops.yaml')

ds_satlas = make_dataset(cfg.satlas.dataset, dino_augm_cfg=cfg.fmow.dino_augm, seed=42)

00:06:48 panopticon.py:28] ChnAttnPatchEmb: id_attn_block: ChnAttnBlockSimple
00:06:48 panopticon.py:104] ChnAttnBlockSimple: norm_input: True, skip_conn: True, norm_output: False, use_layer_scale: False
00:06:48 loaders.py:69] Building dataset "SatlasDataset" ...
00:06:48 augmentations.py:291] ###################################
00:06:48 augmentations.py:292] Using data augmentation parameters:
00:06:48 augmentations.py:293] id: ChnSpatialAugmentationV2
00:06:48 augmentations.py:294] local_crops_number: 4
00:06:48 augmentations.py:295] global_crops_scale: [0.32, 1.0]
00:06:48 augmentations.py:296] local_crops_scale: [0.05, 0.32]
00:06:48 augmentations.py:297] global_crops_size: 224
00:06:48 augmentations.py:298] local_crops_size: 98
00:06:48 augmentations.py:299] global_crops_spectral_size: [6, 13]
00:06:48 augmentations.py:300] local_crops_spectral_size: [3, 6]
00:06:48 augmentations.py:301] global_modes_probs: [0.5, 0.3, 0.2]
00:06:48 augmentations.py:302] local_modes_probs: [0.2, 0

In [69]:
from torch.utils.data import DataLoader
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

dl_fmow = DataLoader(ds_satlas, batch_size=5)

sample = next(iter(dl_fmow))

sample = sample['global_crops_teacher'][0]
# print(sample)

input_0 = dict(imgs = sample['imgs'].to(dtype=torch.float32), chn_ids = sample['chn_ids'], spec_masks = sample['spec_masks'])

# input_0['chn_ids'], input_0['spec_masks']

for k in input_0:
    input_0[k] = input_0[k].to(device)
    torch.cuda.synchronize()

print(input_0['imgs'].device)

pan = pan.to(device)

print(next(pan.parameters()).device)

out = pan(input_0)
out[0].shape

cuda
s2: 9
l9: 11
s2: 9
s1: 2
s1: 2
l9: 11


s2: 9
s1: 2
s1: 2
l9: 11
cuda:0
cuda:0


torch.Size([5, 256, 256])